# NOTE
This notebook doesn't work yet. I have previously written my own version of `lambdify` [here](https://github.com/bjodah/sym/blob/222d1c6a3d1db03c67e8467ff562d3ac0f124616/sym/_sympy_Lambdify.py#L214).
Don't know if that's the path to go, or wait for [next release of numba](https://github.com/numba/numba/issues/2328#issuecomment-303382911).

In this notebook we will use `numba` the increase the performance of our callbacks produced by lambdify in SymPy.

In [ ]:
import json
import numpy as np
import sympy as sp
from scipy2017codegen.odesys import ODEsys
from scipy2017codegen.chem import mk_rsys

The `ODEsys` class and convenience functions from previous notebook (35) has been put in two modules for easy importing. Recapping what we did last:

In [ ]:
watrad_data = json.load(open('../scipy2017codegen/data/radiolysis_300_Gy_s.json'))
watrad = mk_rsys(ODEsys, **watrad_data)
tout = np.logspace(-6, 3, 200)  # close to one hour of operation
c0 = {'H2O': 55.4e3, 'H+': 1e-4, 'OH-': 1e-4}
y0 = [c0.get(symb.name, 0) for symb in watrad.y]

In [ ]:
%timeit yout, info = watrad.integrate_odeint(tout, y0)

so that is the benchmark to beat.

In [ ]:
from numba import njit
watrad_numba = mk_rsys(ODEsys, **watrad_data, lambdify=lambda *args: njit(sp.lambdify(*args, modules="numpy")))
watrad_numba.integrate_odeint(tout, y0)

In [ ]:
%timeit watrad_numba.integrate_odeint(tout, y0)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Just to see that everything looks alright:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
watrad_numba.plot_result(tout, *watrad_numba.integrate_odeint(tout, y0), ax=ax)
ax.set_xscale('log')
ax.set_yscale('log')